In [ ]:
import requests
import base64
import json
import os
import io

In [ ]:
def get_image_objects(image_path):
    # Define the API endpoint
    api_url = "https://vision.googleapis.com/v1/images:annotate"
    
    # Load the image
    with open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

    # Define the request body
    request_body = {
        "requests": [
            {   "image": { "content": encoded_image},
                "features": [{
                            "maxResults": 10,
                            "type": "OBJECT_LOCALIZATION"  }]
            }]}
        # Define the API key
    api_key = ""
    # Make the API request
    response = requests.post(api_url,
                             headers={"Content-Type": "application/json",
                                      "Authorization": "Bearer " + api_key},
                             data=json.dumps(request_body))
    # Parse the response
    response_json = response.json()
    objects = response_json["response"][0]["localizedObjectAnnotations"]
    return objects

In [ ]:
def save_objects_as_images(image_path, objects):
    # Load the image
    from PIL import Image
    original_image = Image.open(image_path)

    for i, obj in enumerate(objects):
        # Get the bounding box coordinates
        x = obj["boundingPoly"]["normalizedVertices"][0]["x"] * original_image.width
        y = obj["boundingPoly"]["normalizedVertices"][0]["y"] * original_image.height
        x2 = obj["boundingPoly"]["normalizedVertices"][2]["x"] * original_image.width
        y2 = obj["boundingPoly"]["normalizedVertices"][2]["y"] * original_image.height

        # Crop the object from the image
        obj_image = original_image.crop((x, y, x2, y2))

        # Save the object as an image
        obj_image.save("object_{}.jpg".format(i))

In [ ]:
if __name__ == "__main__":
    image_path = os.path.abspath('./image01.png')
    #image_path = os.path.abspath('./image02.png')
    objects = get_image_objects(image_path)
    save_objects_as_images(image_path, objects)